In [1]:
%matplotlib inline


# 1) Wipe out your namespace
%reset -f

# 2) Clear Jupyter’s stored outputs (and inputs if you like)
try:
    Out.clear()
except NameError:
    pass

try:
    In.clear()
except NameError:
    pass

# 3) Force Python GC
import gc
gc.collect()

# 4) Free any GPU buffers
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()


import importlib
from libs import params, trades, feats, plots, models_core
importlib.reload(params)
importlib.reload(trades)
importlib.reload(feats)
importlib.reload(plots)
importlib.reload(models_core)

<module 'libs.models_core' from '/workspace/my_models/Trading/_Stock_Analysis_/libs/models_core.py'>

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy  as np
import math
import matplotlib.pyplot as plt

import datetime as dt
import os
from typing import Sequence, List, Tuple, Optional, Union

import torch.nn as nn
import torch.nn.functional as Funct
from torch_lr_finder import LRFinder
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts #, ReduceLROnPlateau
from torch.amp import GradScaler

from tqdm import tqdm

In [3]:
df_feat_sel = pd.read_csv(params.feat_all_csv, index_col=0, parse_dates=True)[params.features_cols_tick + ['close_raw'] + [params.label_col]]
    
df_feat_sel

KeyboardInterrupt: 

In [ ]:
train_loader, val_loader, test_loader, end_times_tr, end_times_val, end_times_te = models_core.model_core_pipeline(
    df             = df_feat_sel,
    look_back      = params.look_back_tick,
    sess_start     = params.sess_start_pred_tick,
    train_prop     = params.train_prop,
    val_prop       = params.val_prop,
    train_batch    = params.hparams["TRAIN_BATCH"],
    num_workers    = params.hparams["NUM_WORKERS"],
    prefetch_factor= params.hparams["TRAIN_PREFETCH_FACTOR"],
    signal_thresh  = params.best_optuna_params["buy_threshold"],
    return_thresh  = params.return_threshold_tick
)

for name, ld, tm in zip(
    ["train","val","test"],
    [train_loader, val_loader, test_loader],
    [end_times_tr, end_times_val, end_times_te]
):
    models_core.summarize_split(name, ld, tm)

In [ ]:
# importlib.reload(params.model_selected) #############

# -----------------------------------------------------------------------------
# Instantiate the ModelClass & move to device
# -----------------------------------------------------------------------------

model = params.model_selected.ModelClass(
    n_feats          = len(params.features_cols_tick),                          
    short_units      = params.hparams['SHORT_UNITS'],    
    long_units       = params.hparams['LONG_UNITS'],     
    dropout_short    = params.hparams['DROPOUT_SHORT'],  
    dropout_long     = params.hparams['DROPOUT_LONG'],   
    # att_heads        = params.hparams['ATT_HEADS'],
    # att_drop         = params.hparams['ATT_DROPOUT'],
    conv_k           = params.hparams['CONV_K'],
    conv_dilation    = params.hparams['CONV_DILATION'],
    # smooth_k         = params.hparams['SMOOTH_K'],
    # smooth_dilation  = params.hparams['SMOOTH_DILATION']
    pred_hidden      = params.hparams['PRED_HIDDEN'],
)
model.to(params.device)  

model

In [ ]:
y_train = np.concatenate([batch[1].cpu().numpy().ravel() for batch in train_loader])
y_val = np.concatenate([batch[1].cpu().numpy().ravel() for batch in val_loader])

# Visualize the true‐signal distributions on train vs. validation
plt.hist(y_train, bins=100, alpha=0.5, label="train true")
plt.hist(y_val,   bins=100, alpha=0.5, label="val true")
plt.xlabel("Signal value")
plt.ylabel("Count")
plt.title("True Signal Distribution: Train vs. Validation")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# import numpy as np
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# import torch

# def full_sanity_check(train_loader, val_loader, model):
#     """
#     Run end-to-end sanity checks on:
#       1) _compute_metrics vs. sklearn
#       2) compute_baselines vs. manual baselines
#       3) eval_on_loader metrics vs. manual flatten+_compute_metrics
#     Raises AssertionError on any mismatch; prints summary if all pass.
#     """
#     # 1) Test _compute_metrics against sklearn on random data
#     rng = np.random.RandomState(42)
#     targs = rng.rand(50)
#     preds = rng.rand(50)
#     m = params.model_selected._compute_metrics(preds, targs)
#     mse = mean_squared_error(targs, preds)
#     assert np.isclose(m["rmse"], np.sqrt(mse), atol=1e-8), "RMSE mismatch"
#     assert np.isclose(m["mae"], mean_absolute_error(targs, preds), atol=1e-8), "MAE mismatch"
#     assert np.isclose(m["r2"], r2_score(targs, preds), atol=1e-8), "R2 mismatch"

#     # 2) Test compute_baselines vs. manual
#     def manual_baselines(loader):
#         nexts, pers_preds, pers_targs = [], [], []
#         for xb, y_r, *_ignore, wd, ts_list, lengths in loader:
#             for i, L in enumerate(lengths):
#                 if L < 1: continue
#                 y = y_r[i, :L].view(-1).cpu().numpy()
#                 nexts.append(y[-1])
#                 if L > 1:
#                     pers_preds.append(y[-2])
#                     pers_targs.append(y[-1])
#         nexts = np.array(nexts, dtype=float)
#         mean_rmse_manual = float(np.sqrt(((nexts - nexts.mean())**2).mean()))
#         if pers_preds:
#             pers_preds = np.array(pers_preds)
#             pers_targs = np.array(pers_targs)
#             pers_rmse_manual = float(np.sqrt(((pers_preds - pers_targs)**2).mean()))
#         else:
#             pers_rmse_manual = float("nan")
#         return mean_rmse_manual, pers_rmse_manual

#     # run on train_loader and val_loader
#     for name, loader in [("TRAIN", train_loader), ("VAL", val_loader)]:
#         base_mean_fn, base_pers_fn = params.model_selected.compute_baselines(loader)
#         mean_m, pers_m = manual_baselines(loader)
#         assert np.isclose(base_mean_fn, mean_m, atol=1e-8), f"{name} mean baseline mismatch"
#         assert np.isclose(base_pers_fn, pers_m, atol=1e-8), f"{name} persistence baseline mismatch"

#     # 3) Test eval_on_loader vs. manual flatten + _compute_metrics
#     def manual_eval(loader):
#         all_preds, all_targs = [], []
#         device = next(model.parameters()).device
#         model.to(device).eval()
#         model.h_short = model.h_long = None
#         prev_day = None
#         with torch.no_grad():
#             for xb, y_reg, *_ignore, wd, ts_list, lengths in loader:
#                 xb, y_reg, wd = xb.to(device), y_reg.to(device), wd.to(device)
#                 B = xb.size(0)
#                 for i in range(B):
#                     prev_day = params.model_selected._reset_states(model, wd[i], prev_day)
#                     W = int(lengths[i])
#                     if W == 0: continue
#                     seqs = xb[i, :W]
#                     raw = model(seqs)
#                     raw_reg = raw[0] if isinstance(raw, (tuple,list)) else raw
#                     if raw_reg.dim()==3 and raw_reg.size(-1)!=1:
#                         raw_reg = model.pred(raw_reg)
#                     elif raw_reg.dim()==2:
#                         raw_reg = model.pred(raw_reg.unsqueeze(0)).squeeze(0)
#                     preds_win = raw_reg.squeeze(-1)[:, -1]
#                     targs_win = y_reg[i, :W].view(-1)
#                     all_preds.extend(preds_win.cpu().tolist())
#                     all_targs.extend(targs_win.cpu().tolist())
#         preds = np.array(all_preds, dtype=float)
#         targs = np.array(all_targs, dtype=float)
#         return params.model_selected._compute_metrics(preds, targs)

#     # run manual_eval on val_loader and compare to eval_on_loader
#     manual_metrics = manual_eval(val_loader)
#     fn_metrics, _      = params.model_selected.eval_on_loader(val_loader, model)
#     for key in ("rmse", "mae", "r2"):
#         assert np.isclose(fn_metrics[key], manual_metrics[key], atol=1e-8), \
#             f"eval_on_loader {key} mismatch: {fn_metrics[key]} vs {manual_metrics[key]}"

#     print("✅ FULL SANITY CHECK PASSED: metrics, baselines, eval align perfectly.")

# full_sanity_check(train_loader, val_loader, model)

In [ ]:
# import numpy as np
# import torch

# def metrics_proof(loader, model):
#     # 1) Re-compute mean‐baseline and persistence‐baseline by hand
#     manual_nexts, manual_deltas = [], []
#     for xb, y_r, *_ignored, wd, ts, lengths in loader:
#         for i, L in enumerate(lengths):
#             if L < 1: 
#                 continue
#             y = y_r[i, :L].view(-1).cpu().numpy()
#             manual_nexts.append(y[-1])
#             if L > 1:
#                 manual_deltas.append(y[-1] - y[-2])

#     nxt = np.array(manual_nexts, dtype=float)
#     deltas = np.array(manual_deltas, dtype=float)
#     mean_rmse_manual = np.sqrt(((nxt - nxt.mean())**2).mean())
#     pers_rmse_manual = np.sqrt((deltas**2).mean())

#     # 2) Call your compute_baselines()
#     mean_rmse_fn, pers_rmse_fn = params.model_selected.compute_baselines(loader)

#     # 3) Flatten preds/targs via eval_on_loader (no clamp)
#     (metrics, preds) = params.model_selected.eval_on_loader(loader, model)
#     # we need the targets too—so do a tiny mod:
#     #   return _compute_metrics(preds,targs), preds, targs
#     # in eval_on_loader, then unpack here:
#     #    metrics, preds, targs = eval_on_loader(...)
#     # For now we’ll reconstruct targs manually:
#     all_targs = []
#     for xb, y_r, *_ignored, wd, ts, lengths in loader:
#         for i, L in enumerate(lengths):
#             if L < 1:
#                 continue
#             all_targs.extend(y_r[i, :L].view(-1).cpu().tolist())
#     targs = np.array(all_targs[: len(preds) ], dtype=float)

#     manual_rmse_flat = np.sqrt(((preds - targs)**2).mean())

#     print(f"Mean baseline:  fn={mean_rmse_fn:.5f}  manual={mean_rmse_manual:.5f}")
#     print(f"Pers baseline:  fn={pers_rmse_fn:.5f}  manual={pers_rmse_manual:.5f}")
#     print(f"Eval RMSE:      fn={metrics['rmse']:.5f}  flat={manual_rmse_flat:.5f}")
#     print("Sample deltas:", deltas[:10])
#     print("Sample preds:", preds[:10])
#     print("Sample targs:", targs[:10])

#     assert np.isclose(mean_rmse_fn, mean_rmse_manual, atol=1e-8)
#     assert np.isclose(pers_rmse_fn, pers_rmse_manual, atol=1e-8)
#     assert np.isclose(metrics['rmse'], manual_rmse_flat, atol=1e-8)
#     print("✅ All RMSEs and baselines match the textbook formula exactly.")

# # Run it on train then val
# metrics_proof(train_loader, model)
# metrics_proof(val_loader,   model)


In [ ]:
importlib.reload(params) #############
importlib.reload(params.model_selected) #############
importlib.reload(models_core) #############

# How many unique trading days does each epoch see?
n_days = len(train_loader.dataset)
print(f"Training sees {n_days} unique trading days per epoch.\n")

print('Using HyperParameters:\n "look_back":', params.look_back_tick, params.hparams)


optimizer = AdamW(
    model.parameters(),
    lr=params.hparams['INITIAL_LR'],
    weight_decay=params.hparams['WEIGHT_DECAY']
)

cosine_sched = CosineAnnealingWarmRestarts(
    optimizer,
    T_0   = params.hparams['T_0'],
    T_mult= params.hparams['T_MULT'],
    eta_min=params.hparams['ETA_MIN']
)

# -----------------------------------------------------------------------------
# Run the custom stateful training loop
# -----------------------------------------------------------------------------
best_val_rmse  = params.model_selected.model_training_loop(
    model               = model,
    optimizer           = optimizer,
    cosine_sched        = cosine_sched,
    scaler              = GradScaler(),
    train_loader        = train_loader,
    val_loader          = val_loader,
    max_epochs          = params.hparams['MAX_EPOCHS'],
    early_stop_patience = params.hparams['EARLY_STOP_PATIENCE'],
    clipnorm            = params.hparams['CLIPNORM'],
    # cls_loss_weight     = params.hparams['CLS_LOSS_WEIGHT'],
    # smooth_alpha        = params.hparams['SMOOTH_ALPHA'],
    # smooth_beta         = params.hparams['SMOOTH_BETA'],
    # smooth_delta        = params.hparams['SMOOTH_DELTA'],
    # diff1_weight        = params.hparams['DIFF1_WEIGHT'],
    # diff2_weight        = params.hparams['DIFF2_WEIGHT'],
)
